# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [15]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

2024-08-01 07:22:57,079 - 139628944219008 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

- Set the inputs for the execution of the crew.

In [16]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [17]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of analyzing market data for AAPL to the Data Analyst in order to identify trends and predict market movements.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for AAPL using statistical modeling and machine learning to identify trends and predict market movements", "context": "The task involves analyzing historical and real-time market data for AAPL, applying statistical modeling techniques, and using machine learning algorithms to make predictions about market movements."}


> Entering new CrewAgentExecutor chain...
I need to gather historical and real-time market data for AAPL, apply statistical mode

Final Answer: Apple Inc. (AAPL) is currently trading at $222.08, with a 1.50% increase. The market cap is $3.405 trillion, and the previous close was $218.80. The stock has a P/E ratio of 34.48 and an EPS of 6.44. The company designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. They also offer various services such as Apple Arcade, Apple Fitness+, Apple Music, Apple News+, Apple TV+, Apple Card, and Apple Pay. Apple Inc. serves consumers, small and mid-sized businesses, education, enterprise, and government markets. They distribute third-party applications through the App Store and sell products through retail and online stores, as well as third-party cellular network carriers, wholesalers, retailers, and resellers. The company was founded in 1976 and is headquartered in Cupertino, California.

> Finished chain.
 

Apple Inc. (AAPL) is currently trading at $222.08, with a 1.50% increase. The market cap is $3.405 trillion, an

Thought: I have gathered valuable insights from the Data Analyst Agent on AAPL trading strategies.

Final Answer: 
I found several resources providing insights on trading strategies for AAPL:

1. Title: Apple Trading Strategies & Insights - LinkedIn
Link: [Apple Trading Strategies & Insights - LinkedIn](https://www.linkedin.com/pulse/apple-trading-strategies-insights-quantifiedstrategies-apmpc)
Snippet: Ready to make some serious cash with AAPL stocks? Then you need to know the ins and outs of AAPL stock trading strategies, including price action analysis and using call options.

2. Title: AAPL (Apple Inc.) Technical Analysis: Insights for Traders - Vestinda
Link: [AAPL (Apple Inc.) Technical Analysis: Insights for Traders - Vestinda](https://www.vestinda.com/academy/aapl-apple-inc-technical-analysis-insights-for-traders)
Snippet: Strategic Approach to Analyzing Apple's Technicals · Start by gathering the necessary data for AAPL, including historical prices, volume, and financial state

Final Answer: 
I found various resources discussing approved trading strategies for AAPL that incorporate technical and price action analysis, consider call options, and analyze AAPL's fundamentals. Here are some of the key insights:

- Technical analysis incorporating earnings is crucial for developing successful trading strategies to buy Apple stock and make money.
- Different options trading strategies such as long calls, long puts, covered calls, protective puts, and straddles can be considered for AAPL.
- Technical analysis tools can help traders identify potential trading opportunities based on past price action.
- Day trading options for income can be a profitable strategy for AAPL.
- The Anchored VWAP indicator is a popular technical analysis tool used by traders and investors to analyze price action.

For more in-depth information and specific trading strategies, please refer to the provided links for a comprehensive understanding of approved trading strategies for AAPL.

> Fi

Thought: I now can give a great answer
Final Answer: 
After conducting a search on the internet for AAPL trading strategies, I found various resources that provide insights into different trading approaches for Apple Inc. These strategies include options trading, day trading, swing trading, and long-term investing. It is crucial to consider the company's fundamentals, technical characteristics, and the investor's risk tolerance when developing a trading strategy for AAPL. Additionally, utilizing tools like payout diagrams, chart patterns analysis, and trend moving strategies can help optimize trading performance. It is recommended to diversify trading strategies to mitigate risks and capitalize on potential price movements in AAPL stock.

> Finished chain.
 

After conducting a search on the internet for AAPL trading strategies, I found various resources that provide insights into different trading approaches for Apple Inc. These strategies include options trading, day trading, swing t

- Display the final result as Markdown.

In [18]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for AAPL Trading Strategies and Execution Plans:

Introduction:
As the Trading Strategy Developer for AAPL, I have analyzed the approved trading strategies focusing on technical and price action analysis to identify entry and exit points. Call options have been considered to capitalize on potential price movements. To further enhance the trading performance and align with the user's risk tolerance level, a thorough risk analysis is essential.

Risk Analysis:
1. Market Risk: The overall market conditions can impact the price movements of AAPL stocks. Economic indicators, geopolitical events, and market sentiment can influence the stock's performance. Mitigation: Diversification of the trading strategies and closely monitoring market trends can help mitigate market risks.

2. Company Risk: AAPL's market position, product offerings, and financial statements are vital factors that can affect trading performance. Changes in leadership, product recalls, or financial instability can pose risks. Mitigation: Conducting detailed fundamental analysis, staying updated on company news, and setting stop-loss orders can mitigate company-specific risks.

3. Technical Risk: Technical analysis tools and indicators may provide false signals leading to incorrect trading decisions. Inaccurate price action analysis can result in losses. Mitigation: Combining multiple technical indicators, backtesting strategies, and utilizing risk management techniques can reduce technical risks.

4. Execution Risk: Incorrect execution of trades, delays in order fulfillment, or technical issues with trading platforms can impact trading outcomes. Poorly timed trades can lead to missed opportunities or losses. Mitigation: Implementing limit orders, setting trading alerts, and using reliable trading platforms can minimize execution risks.

5. Options Risk: Trading call options can amplify gains but also increase potential losses. Lack of understanding options trading strategies can lead to substantial risks. Mitigation: Educating on options trading, implementing risk management strategies, and using options as part of a diversified trading approach can mitigate options risks.

Recommendations:
- Continuously monitor market conditions and company news to stay informed.
- Utilize a combination of technical and fundamental analysis for informed trading decisions.
- Implement risk management techniques such as stop-loss orders and position sizing.
- Diversify trading strategies to reduce overall risk exposure.
- Stay updated on options trading strategies and utilize them effectively within the trading plan.

Conclusion:
By conducting a thorough risk analysis and implementing mitigation strategies, the trading performance for AAPL can be optimized while aligning with the user's risk tolerance level. It is crucial to stay vigilant, adapt to changing market conditions, and make informed trading decisions to achieve successful outcomes in AAPL trading strategies.